In [4]:
import pandas as pd
import re
from string import digits

In [2]:
def take_text(fin, fout):
    f = open(fin, 'r', encoding = 'utf-8')
    twi = []
    cont = []
    l = f.readlines()
    for i in l:
        i = i.replace('"\n', '')
        if i[0] == '"':
            lst = i[1:].split('";"')
        else:
            lst = i.split('";"')
        if len(lst) == 12:
            twi.append(lst)
        elif cont == []:
            cont += lst
        elif '\n' in cont[-1] and lst != []:
            c = cont[-1]
            cont = cont[:-1]
            cont.append(c+lst[0])
            cont += lst[1:]
            if len(cont) == 12:
                twi.append(cont)
                cont = []
    f.close()
    
    only_twits = []
    for line in twi:
        only_twits.append(line[3])
    
    f = open(fout, 'w', encoding = 'utf-8')
    for i in only_twits:
        f.write(i+'\n')
    f.close()

In [3]:
take_text('positive.csv', 'positive_text.txt')
take_text('negative.csv', 'negative_text.txt')

In [123]:
def take_emojis():
    f = open('emojis.txt', 'r', encoding='utf-8')
    emo = [line.strip() for line in f]
    emojis = '[' + '|'.join(emo) + ']'
    f.close()
    return emojis

In [124]:
emojis = take_emojis()

In [136]:
def smiles_emo(fin, fout, emojis):
    fi = open(fin, 'r', encoding = 'utf-8')
    l = fi.read()
    fi.close()
    l = re.sub(r"((https?:\/\/|www)|\w+\.(\w{2-3}))([\w\!#$&-;=\?\-\[\]~]|%[0-9a-fA-F]{2})+", '', l) #убираем ссылки
    smiles = re.findall(r"(?:[<>]?[:;=8x][ \-o\*\']?[\)\]\(\[dDpP3*/\:\}\{@\|\\]|[\)\]\(\[dDpPсСрР3/\:\}\{@\|\\][ \-o\*\']?[:;=8][<>]?)", l)
#    l = re.sub(r"(?:[<>]?[:;=8x][ \-o\*\']?[\)\]\(\[dDpP3*/\:\}\{@\|\\]|[\)\]\(\[dDpPсСрР3/\:\}\{@\|\\][ \-o\*\']?[:;=8][<>]?)", '', l) # убираем смайлики
    emo = re.findall(emojis, l)
    l = re.sub(emojis, '', l)                                            # убираем эмоджи
    fo = open(fout, 'w', encoding = 'utf-8')
    fo.write(l)
    fo.close()
    return smiles, emo

In [137]:
pos_smiles, pos_emo = smiles_emo('positive_text.txt', 'pos_without_emo.txt', emojis)
neg_smiles, neg_emo = smiles_emo('negative_text.txt', 'neg_without_emo.txt', emojis)

In [141]:
pos_emo

['2',
 '9',
 '1',
 '2',
 '2',
 '9',
 '0',
 '0',
 '3',
 '1',
 '7',
 '8',
 '9',
 '#',
 '#',
 '4',
 '3',
 '™',
 '1',
 '6',
 '1',
 '0',
 '1',
 '2',
 '1',
 '4',
 '6',
 '1',
 '3',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '6',
 '9',
 '3',
 '5',
 '1',
 '8',
 '3',
 '2',
 '0',
 '0',
 '3',
 '8',
 '1',
 '5',
 '2',
 '0',
 '5',
 '9',
 '2',
 '0',
 '1',
 '0',
 '1',
 '1',
 '8',
 '#',
 '8',
 '0',
 '3',
 '5',
 '3',
 '6',
 '0',
 '#',
 '#',
 '7',
 '5',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '#',
 '#',
 '#',
 '1',
 '8',
 '7',
 '0',
 '2',
 '1',
 '8',
 '0',
 '3',
 '1',
 '3',
 '#',
 '5',
 '0',
 '4',
 '3',
 '9',
 '0',
 '4',
 '9',
 '2',
 '7',
 '9',
 '1',
 '0',
 '1',
 '#',
 '#',
 '#',
 '2',
 '5',
 '1',
 '2',
 '3',
 '3',
 '1',
 '0',
 '0',
 '0',
 '#',
 '9',
 '5',
 '#',
 '6',
 '1',
 '2',
 '0',
 '#',
 '#',
 '1',
 '#',
 '1',
 '5',
 '1',
 '9',
 '3',
 '9',
 '4',
 '0',
 '9',
 '1',
 '8',
 '0',
 '0',
 '1',
 '2',
 '0',
 '3',
 '0',
 '7',
 '2',
 '0',
 '1',
 '3',
 '6',
 '#',
 '#',
 '#',
 '4',
 '5',
 '7',
 '2',
 '4',
 '3',
 '1',
 '0'

In [98]:
def normalize(fin, fout):
    fi = open(fin, 'r', encoding = 'utf-8')
    l = fi.read()
    fi.close()                                          
    l = re.sub(r"(?:@\w+)", '', l)                                        # убираем пользователя
    l = re.sub(r"[\w.+-]+@[\w-]+\.(?:[\w-]\.?)+[\w-]", '', l)             # убираем email
    l = re.sub(r"(?:I'm at .*? in .*?(?: w/ @[\w])?|\(@ .*?\))", '', l)   # убираем геолокации
    l = l.replace('RT', '')                                              # убираем RT    
    l = l.translate(str.maketrans('', '', digits))                        # убираем все числа
#    l = l.translate(dict.fromkeys(map(ord, ',!.;?%=&*$#:"/\|()[]{}')))             # убираем знаки препинания
    l = l.replace('#', '')
    lower_up = [m.start() for m in re.finditer(r"[a-zа-яё]{1}[A-ZА-ЯЁ]{1}", l)]   # добавляем пробел между lower- и uppercase HiWorld => Hi World 
    print(len(lower_up))
    for i in reversed(lower_up):
        l = l[:i+1] + ' ' + l[i+1:]
    l = l.lower()                                                         # уменьшаем регистр у всех букв  
    
    fo = open(fout, 'w', encoding = 'utf-8')
    fo.write(l)
    fo.close()

In [100]:
normalize('pos_without_emo.txt', 'norm_pos_text.txt')
normalize('neg_without_emo.txt', 'norm_neg_text.txt')

2572
lu


In [139]:
from collections import Counter

data_pos = Counter(pos_smiles)
data_pos.most_common()

[(':)', 19653),
 (':D', 13104),
 (': @', 4861),
 (';)', 4026),
 (':-)', 3050),
 ('=)', 2094),
 (':*', 2049),
 (':3', 927),
 (':-D', 616),
 ('с:', 425),
 (':**', 412),
 ('d:', 408),
 ('p:', 359),
 (';-)', 352),
 ('3:', 328),
 (';3', 248),
 ('8:', 231),
 ('р :', 217),
 ('с :', 212),
 ('р:', 194),
 ('8 @', 189),
 ('x:', 176),
 ('):', 141),
 ('D:', 139),
 ('p;', 130),
 (': )', 124),
 ('83', 120),
 ('x @', 93),
 ('xD', 92),
 (':*:', 87),
 ('38', 86),
 ('::', 72),
 ('xp', 70),
 (';D', 67),
 ('С:', 67),
 (';:', 67),
 ('@8', 62),
 (') :', 56),
 (':*)', 52),
 ('3 :', 49),
 ('8)', 49),
 (': :', 45),
 (':-*', 36),
 (':/', 31),
 ('D :', 28),
 ('с 8', 28),
 (': D', 26),
 ('; )', 26),
 ('8-)', 25),
 (';*', 25),
 ('=D', 24),
 ('с ;', 24),
 ('; 3', 24),
 ('x)', 22),
 (': 3', 21),
 ('po:', 20),
 ('=*', 19),
 ('P:', 19),
 ('Р :', 19),
 (':d', 18),
 (':P', 17),
 ('xd', 16),
 ('xP', 16),
 ('р;', 16),
 ('=**', 16),
 ('xo:', 15),
 ('do:', 14),
 (': *', 14),
 (': 8', 14),
 (');', 14),
 ('с;', 13),
 ('с =', 1

In [140]:
data_neg = Counter(neg_smiles)
data_neg.most_common()

[(':(', 31369),
 (';(', 3449),
 (': @', 2580),
 ('=(', 2021),
 (':-(', 2000),
 (':|', 755),
 (':3', 489),
 (':-/', 443),
 ('с:', 359),
 ('3:', 294),
 ('d:', 262),
 ('p:', 231),
 ('x:', 208),
 (': (', 206),
 ('8:', 183),
 ('р :', 169),
 ('с :', 148),
 (';:', 125),
 (';-(', 122),
 ('38', 121),
 ('р:', 111),
 (';3', 108),
 (':/', 99),
 ('(:', 97),
 ('p;', 95),
 ('83', 88),
 ('D:', 80),
 ('8 @', 79),
 ('8(', 71),
 ('; (', 66),
 ('x @', 54),
 ('/:', 50),
 ('::', 45),
 ('С:', 43),
 ('(;', 41),
 ('xp', 40),
 (':-D', 37),
 ('с 8', 36),
 (': :', 36),
 ('do:', 30),
 ('@8', 28),
 ('( :', 26),
 ('(-8', 25),
 (';D', 25),
 ('P:', 23),
 ('3 :', 18),
 (':-*', 18),
 ('po:', 17),
 ('(=', 17),
 ('=-(', 17),
 ('р ;', 17),
 (';*', 17),
 ('=D', 16),
 ('с;', 14),
 ('р =', 13),
 ('р;', 13),
 (': 3', 13),
 ('d :', 12),
 ('С :', 12),
 ('с ;', 12),
 ('(8', 12),
 ('8 :', 11),
 (': *', 11),
 ('=*(', 10),
 ('d8', 10),
 ('x 3', 10),
 ('=*', 10),
 ('xo:', 9),
 ('x-(', 9),
 ('с =', 9),
 ('x3', 8),
 (';/', 8),
 ('=**',